# Install and import packages

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install xformers
!pip install -U sentence-transformers

In [ ]:
! pip install jsonlines
! pip install fast_ml --quiet
! pip install transformers
! pip install nltk
! python -m nltk.downloader all
! pip install unidecode

In [ ]:
from unidecode import unidecode
import nltk
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
import string
import tensorflow as tf
import transformers
from textblob import TextBlob
import os
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoModelForSequenceClassification,AutoTokenizer,pipeline

In [ ]:
# Display all column values for validation purpose
pd.set_option('max_colwidth', None)

# Load benchmark data

In [ ]:
# Load data
df_total = pd.read_parquet('/content/drive/MyDrive/Dissertation/Data/df.parquet')
# One-shot sample
df_label_unique_sample = df_total.groupby('label_cat', group_keys=False).apply(lambda df: df.sample(1))
# Create label mapping
label = list(df_label_unique_sample['label'])
id = list(df_label_unique_sample['label_cat'])
label_to_id = dict(zip(label,id))
id_to_label = dict(zip(id,label))

In [ ]:
# Benchmark sentence similarity data
binary_classification_data = pd.read_excel('/content/drive/MyDrive/Dissertation/Label_Classification JC comments.xlsx')
binary_classification_data['Label_text'] = binary_classification_data['Label'].map(id_to_label)

In [ ]:
# Split into Yes only lists and variables with Yes&No text
Label_yes = [20,0,21,25,4,8,10,5,23,19,22,28,27,30]
Label_yes_no = [3,6,7,9,18]
total_label = list(range(0,31))
total_second_stage_label  = Label_yes + Label_yes_no
total_first_stage_label  = [i for i in total_label if i not in total_second_stage_label]

# First-stage classification

## Define functions

In [ ]:
model_neo = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Dissertation/GPT_data/GPT_Neo_model/augmented_gptneo_100")
tokenizer_bert = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# Create a pipeline
classifier_neo = pipeline("text-classification", model=model_neo,tokenizer=tokenizer_bert)

In [ ]:
# Define a function that performs text classification
def open_file(path):
  with open(path) as f:
    lines = f.readlines()
    # Remove all \n only elements
    lines = [i for i in lines if i != '\n']
    # Remove all lines that is less than 15 characters which is the new line or section
    lines = [i for i in lines if len(i)>15]
    # Remove \n for each elemtn in the text file
    lines = [i[:-1] for i in lines]
  return lines

In [ ]:
# Preprocess the dataset for word embedding
def pre_process(text):
    # convert input corpus to lower case.
    text = text.lower()
    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    text = " ".join([i for i in word_tokenize(text) if i not in stopset])
    return text

# Lemmatization and spell check
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

# Word Tokenisationb & Lemmatization
def word_token(sentence):
  words = word_tokenize(sentence)
  for w in range(0,len(words)):
    words[w]=lemmatizer.lemmatize(words[w])
    w+=1
  words = ' '.join(words)
  return words

#Pass the processed text into the pipeline
def modelling(lines,classifier):
  processed_lines = []
  for i in range(len(lines)):
    lines_processed = pre_process(lines[i])
    line_token = word_token(lines_processed)
    result = classifier(line_token )[0]
    result['Text'] = line_token
    processed_lines.append(result)
    i+=1
  return processed_lines

In [ ]:
# Find consecutive lines of text with the same labelling
def consecutive_check(file):
  consecutive_check = []
  for i in range(0,len(file)-1):
    curr_label = list(file[i].values())[0]
    next_label = list(file[i+1].values())[0]
    if curr_label == next_label:
      consecutive_check.append(1)
    else:
      consecutive_check.append(0)
    i+=1
  return consecutive_check

# Find index of those consecutive elements
def consecutive_check_index(consecutive_check):
  consecutive_check_index = []
  for i in range(len(consecutive_check)):
    if consecutive_check[i]==1:
      consecutive_check_index.append(i)
  return consecutive_check_index

# Find the index of consecutive elements stops
def non_consecutive_check_index(consecutive_check_index):
  non_consecutive_check_index = []
  for i in range(0,len(consecutive_check_index)-1):
    if consecutive_check_index[i+1] - consecutive_check_index[i]!=1:
      non_consecutive_check_index.append(i+1) # Adjust the end index
  return non_consecutive_check_index


# Insert starting index
def segment(non_consecutive_check_index1,consecutive_check_index):
  segment_consecutive_check = []
  for i in range(len(non_consecutive_check_index1)-1):
    curr = non_consecutive_check_index1[i]
    next = non_consecutive_check_index1[i+1]
    list_consecutive = consecutive_check_index[curr:next]
    segment_consecutive_check.append(list_consecutive)
    max_indices = max(segment_consecutive_check[i])
    mapping_indices = [max_indices+1, max_indices+2]
    for x in mapping_indices:
      segment_consecutive_check[i].append(x)
    i+=1
  return segment_consecutive_check


def text_concat(segment_consecutive_check,pre_processed_files):
  # Text Concatenation dictionary
  for i in range(len(segment_consecutive_check)):
    dic={}
    list_dir = segment_consecutive_check[i]
    min_dir = min(list_dir)
    max_dir = max(list_dir) # Adjust for mapping
    List_text = pre_processed_files[min_dir : max_dir]
    Text = ' '.join([d.get('Text') for d in List_text])
    Label = pre_processed_files[min_dir]['label']
    Score = max([d.get('score') for d in List_text]) # Use the average score
    dic['Text'] = Text
    dic['label'] = Label
    dic['score'] = Score
    pre_processed_files.append(dic)
    i+=1

def removal(segment_consecutive_check,pre_processed_files):
  # remove original individual text from extraction
  segment_consecutive_check_reverse = sorted(segment_consecutive_check, reverse=True)
  for i in range(len(segment_consecutive_check_reverse)):
    list_dir = segment_consecutive_check[i]
    min_dir = min(list_dir)
    max_dir = max(list_dir)
    del pre_processed_files[min_dir:max_dir]
    i+=1

def concatenate_original(lines,segment_consecutive_check): # Concatenate processed lines
  for i in range(len(segment_consecutive_check)):
    list_dir = segment_consecutive_check[i]
    min_dir = min(list_dir)
    max_dir = max(list_dir) # Adjust for mapping
    List_text = lines[min_dir : max_dir]
    Text = ' '.join([str(item) for item in List_text])
    lines.append(Text)
    i+=1
  removal(segment_consecutive_check,lines)
  return lines

In [ ]:
# Return the best results for each label and find unprocessed text in original file. All has been stored into one pandas dataframe
def result_to_table(procssed_list,orig_list):
  # Maximum score of each label
  label = [d.get('label') for d in procssed_list]
  text = [d.get('Text') for d in procssed_list]
  score = [d.get('score') for d in procssed_list]
  data = {'Text': text, 'Label':label , 'score': score}
  # Get the largest score of sample with the same label
  sample = pd.DataFrame(data)
  # Add an index column
  sample['Index_ref'] = sample.index
  max_table = sample[sample.groupby('Label')['score'].transform(max) == sample['score']]
  max_table.sort_values(by=['Index_ref'],inplace=True)
  # Get the +-2 sentences for each label
  index_list = sorted(list(max_table['Index_ref']))
  #return index_list
  text_all= []
  label_all = []
  for i in index_list:
    text_range = orig_list[i-3:i+3]
    text_all.append(text_range)
  max_table['Orig_Text'] = text_all
  max_table['Label_cat'] = max_table['Label'].map(label_to_id)
  max_table.drop(columns = ['Index_ref','Text'],axis=1,inplace=True)
  # Remove Duplicate label
  max_table_final = max_table.drop_duplicates(subset=['Label','Label_cat'], keep='first')
  return max_table_final

# Second-stage Binary Classification

## Define functions

In [ ]:
#Import sentence similarity model
model_infloat = SentenceTransformer('intfloat/e5-small-v2')

In [ ]:
def sent_similarity(model,label_cat, label,text,  target_sent,threshold):
  #text_extract['Label_cat'] = text_extract['Label'].map(label_to_id)
    # Text from the text extraction output
  # Label of this row of extraction
  label_extract = label_cat
  label_extract_text = label
  df = target_sent[target_sent['Label']==label_extract]
  # check whether the label belongs to Label_yes only or Label_yes_no
  if label_extract in Label_yes:
  # Target sentence from the Yes-only label
    target_sent_yes = list(df['Sentence (for Yes)'])
    text_score = sent_sim_func(model,text,target_sent_yes,label_extract_text)
    # Return binary classification over threshold
    score = text_score['Score']
    Label = text_score['Label']
    if score >=threshold:
        dic = {'Label':Label, 'Result': 'Yes'}
    else:
        dic = {'Label':Label, 'Result': 'No'}
  else: # Belongs to yes/no label
      yes_sent = list(df['Sentence (for Yes)'])
      no_sent = list(df['Sentence (for No)'])
      clean_no_sent = [x for x in no_sent if str(x) != 'nan']
      total_sent = yes_sent + clean_no_sent
      text_score = sent_sim_func(model,text,total_sent,label_extract_text)
      score = text_score['Score']
      Label = text_score['Label']
      if score >=threshold:
        # Check whther it is a Yes or No score
        if text_score['target_text'] in yes_sent:
          dic = {'Label':Label, 'Result': 'Yes'}
        else:
          dic = {'Label':Label, 'Result': 'No'}
      else:
        dic = {'Label':Label, 'Result': 'None'}
  return dic

In [ ]:
# Define a general function to find simialrity for labels that exists from the first classification model
def sent_sim_func(model,source_sent,target_sent, label):
  max_score = []
  for x in range(len(source_sent)):
    for y in range(len(target_sent)):
        embedding_1 = model.encode(source_sent[x], convert_to_tensor=True)
        embedding_2 = model.encode(target_sent[y], convert_to_tensor=True)
        score = float(util.pytorch_cos_sim(embedding_1, embedding_2)[0][0])
        target_text = target_sent[y]
        source_text = source_sent[x]
        dic = {'Text_extracted':source_text,'target_text':target_text,
               'Score':score, 'Label': label}
        max_score.append(dic)
        y+=1
    x+=1
  max_score_all = [d.get('Score') for d in max_score]
  max_score_index = max_score_all.index(max(max_score_all))
  max_score_final = max_score[max_score_index]
  return max_score_final

# First_stage_second_stage combination Function

In [ ]:
# Define a function to decide which one to go through second-stage classification
def first_to_second(dataframe,orig_file): # dataframe is the output from the first classification model, orig_file is the opened original lease txt file
# Create a final table with Label and score
# Filter out rows with empty text
  dataframe = dataframe[dataframe['Orig_Text'].str.len() > 0]
  df_all = pd.DataFrame(columns = ['Label','Result'])
# Find missing label
  exist_label = list(dataframe['Label_cat'])
  label_first = [i for i in exist_label if i in total_first_stage_label]
  label_second = [i for i in exist_label if i in total_second_stage_label]
  label_second_missing = [i for i in total_second_stage_label if i not in label_second]
  label_first_missing = [i for i in total_first_stage_label if i not in label_first]
# Construct the pipline to merge first and second stage classification
  for i in range(len(dataframe)):
    label = dataframe.iloc[i]['Label']
    label_cat = dataframe.iloc[i]['Label_cat']
    score = dataframe.iloc[i]['score']
    text = dataframe.iloc[i]['Orig_Text']
    # Append labels to the dataframe if labels does not need to be binary classified
    if label_cat in label_first:
      if score>=0.8:
        data = {'Label': label, 'Result': text}
      elif score<=0.8:
        data = {'Label': label, 'Result': 'None'}
    else: # Labels need to be input into second stage classification
      if score>=0.9:
        data = sent_similarity(model_infloat,label_cat,label,text,binary_classification_data,0.85)
      else:
        data = sent_similarity(model_infloat,label_cat,label,orig_file,binary_classification_data,0.85)
    df_all = df_all.append(data,ignore_index=True)
    i+=1
# Construct a pipeline to treat missing data
# Missing Labels for first stage modelling
  if len(label_first_missing)>0:
    for i in range(len(label_first_missing)): # Labels from classification
      label_miss_first = id_to_label[label_first_missing[i]]
      data = {'Label': label_miss_first, 'Result': 'None'}
      df_all = df_all.append(data,ignore_index=True)
      i+=1
  if len(label_second_missing)>0:
    for i in range(len(label_second_missing)):
      label_cat_missing = label_second_missing[i]
      label_missing = id_to_label[label_cat_missing]
      data = sent_similarity(model_infloat,label_cat_missing,label_missing,orig_file,binary_classification_data,0.85)
      df_all = df_all.append(data,ignore_index=True)
      i+=1
  return df_all

In [ ]:
def append_result(dataframe, target_df):
  df_transposed = dataframe.T
  dict_example = {}
  for i in range(len(dataframe)):
    label = df_transposed.iloc[:,i]['Label']
    result = df_transposed.iloc[:,i]['Result']
    dict_example[label] = result
    i+=1
  target_df = target_df.append(dict_example,ignore_index=True)
  return target_df

# Concatenation of classification functions

In [ ]:
def pipeline(filepath,classifier,output_dataframe):
  # First-stage classification
  file_name = filepath.split('/')[-1][:-4]
  lines = open_file(filepath)
  pre_processed_f = modelling(lines,classifier)
  consecutive_c = consecutive_check(pre_processed_f)
  consecutive_c_i = consecutive_check_index(consecutive_c)
  non_consecutive_index = non_consecutive_check_index(consecutive_c_i)
  segment_consecutive_check = segment(non_consecutive_index,consecutive_c_i)
  text_concat(segment_consecutive_check,pre_processed_f)
  removal(segment_consecutive_check,pre_processed_f)
  orig_file = concatenate_original(lines,segment_consecutive_check)
  final_df = result_to_table(pre_processed_f,orig_file)
  # Save first-stage classification to folder for reference
  final_df.to_pickle(f'/content/drive/MyDrive/Dissertation/Experiment_SQL/First_stage_output/{file_name}.pkl')
  # Second-stage classification (for applicable labels)
  result= first_to_second(final_df,lines)
  # Save second-stage classification to folder for reference
  result.to_pickle(f'/content/drive/MyDrive/Dissertation/Experiment_SQL/First+Second_stage_output/{file_name}.pkl')
  # Output pivot
  final_cleaned_df = append_result(result,output_dataframe)
  return final_cleaned_df

# Post-output transformation Function

In [ ]:
# Define a function that assist label: premise/use classification (primary key)
def open_file_premise(path):
  with open(path, encoding = 'utf-8') as f:
    lines = f.readlines()
    # Remove all \n only elements
    lines = [i for i in lines if i != '\n']
    # Remove all lines that is less than 15 characters which is the new line or section
    lines = [i for i in lines if len(i)>10]
    # Remove \n for each elemtn in the text file
    lines = [i[:-1] for i in lines]
    # Select first 50 lines
    lines = lines[:50]
  return lines


# Find label: premise if the score from text classification is low
def find_premise(filepath):
  lines = open_file_premise(filepath)
  for i in lines:
    if (('relating to' in i.lower()) or ('related to' in i.lower())):
        anc_index = lines.index(i)
        line_extract = lines[anc_index+1:anc_index+3]
        break
    elif 'lr4' in i.lower():
        anc_index = lines.index(i)
        line_extract = lines[anc_index+1:anc_index+4]
        break
    elif 'property known' in i.lower():
        anc_index = lines.index(i)
        line_extract = lines[anc_index:anc_index+2]
        break
    elif 'premise' in i.lower():
        anc_index = lines.index(i)
        line_extract = lines[anc_index:anc_index+2]
        break
  return line_extract

# Find label: use if the score from text classification is low
def find_use(filepath):
  lines = open_file(filepath)
  for i in lines:
    if 'permitted use' in i.lower():
        anc_index = lines.index(i)
        line_extract = lines[anc_index:anc_index+3]
        break
    elif 'use as' in i.lower():
        anc_index = lines.index(i)
        line_extract = lines[anc_index:anc_index+3]
        break
    elif 'purpose other' in i.lower():
        anc_index = lines.index(i)
        line_extract = lines[anc_index+1:anc_index+3]
        break
    else:
        line_extract = 'None'
  return line_extract

In [ ]:
# Trimming
def remove_list_df(df):
  df = df.astype(str)
  df = df.applymap(lambda x: x[2:len(x)-2] if len(x)>=15 else x) # Avoid removing 'yes', 'no','others','none' value
  # Remove some special characters
  remove_character= ["'", '"', '“', '”',':','~','[',']']
  for i in remove_character:
    df= df.applymap(lambda x: x.replace(i, ''))
  return df

In [ ]:
# Post-output transformation on text data
def post_output_transformation(dataframe,filepath_collection):
    df_cleaned_trimmed = dataframe.copy()
    rent_review_method = []
    rent_review_date = []
    comm_date = []
    rent_comm_date = []
    term_period = []
    Landlord = []
    Tenant = []
    Premise = []
    Use = []
    for rows in range(len(df_cleaned_trimmed)):
      # Rent Review method clean
      test = []
      df_review = df_cleaned_trimmed['Rent_Review_method'][rows]
      for i in range(len(df_review)):
        text_review_method = df_review[i].lower()
        if 'open market' in text_review_method:
          test.append('Open Market')
        else:
          test.append('Other')
        i+=1
      for i in test:
        if 'Open Market' in i:
          test = 'Open Market'
        else:
          test = 'Other'
      df_cleaned_trimmed.loc[rows,'Rent_Review_method'] = test
      # Rent review date clean
      test_1 = []
      df_review_date = df_cleaned_trimmed['Remain_review_date'][rows]
      for i in range(len(df_review_date)):
        text_review_date = df_review_date[i].lower()
        if 'review date' in text_review_date:
          test_1.append(text_review_date)
        i+=1
      if test_1 == []:
        test_1.append('None')
      df_cleaned_trimmed.loc[rows,'Remain_review_date'] = test_1
      # Commencement date
      test_3 = []
      df_review_comm_date = df_cleaned_trimmed['Commence_date'][rows]
      for i in range(len(df_review_comm_date)):
        text_comm_date = df_review_comm_date[i].lower()
        text_review_date_cross = df_review_date[i].lower()
        if df_review_comm_date != 'None':
          if 'commencement date' in text_comm_date:
            test_3.append(text_comm_date)
          elif 'commencement date' in text_review_date_cross:
            test_3.append(text_review_date_cross)
        elif 'commencement date' in text_review_date_cross:
          test_3.append(text_review_date_cross)
        i+=1
      if test_3 == []:
        test_3.append('None')
      df_cleaned_trimmed.loc[rows,'Commence_date'] = test_3

      # Rent commencement date
      test_4 = []
      df_rent_date = df_cleaned_trimmed['Rent_Commence_date'][rows]
      for i in range(len(df_rent_date)):
        text_rent_comm_date = df_rent_date[i].lower()
        if 'rent commencement' in text_rent_comm_date:
          test_4.append(text_rent_comm_date)
        i+=1
      if test_4 == []:
        test_4.append('None')
      df_cleaned_trimmed.loc[rows,'Rent_Commence_date'] = test_4

      # term_period
      test_5 = []
      df_term_period = df_cleaned_trimmed['Term_period'][rows]
      for i in range(len(df_term_period)):
        text_term_period = df_term_period[i].lower()
        if (('years from' in text_term_period) or ('years commencing' in text_term_period)):
          test_5.append(text_term_period)
        i+=1
      if test_5 == []:
        test_5.append('None')
      df_cleaned_trimmed.loc[rows,'Term_period'] = test_5

      # Parties
      landlord_test = []
      tenant_test = []
      df_parties = df_cleaned_trimmed['Parties'][rows]
      for i in range(len(df_parties)):
        text_parties = df_parties[i].lower()
        if (('(1)' in text_parties) or ('landlord' in text_parties)):
          landlord_test.append(text_parties)
        elif (('(2)' in text_parties) or ('tenant' in text_parties)):
          tenant_test.append(text_parties)
        i+=1
      if tenant_test == []:
        tenant_test.append('CVS Limited') # CVS Leases
      if landlord_test == []:
        tenant_test.append('None')
      Landlord.append(landlord_test)
      Tenant.append(tenant_test)

      # Use
      use_test = []
      df_use = df_cleaned_trimmed['Use'][rows]
      for i in range(len(df_use)):
        text_use = df_use[i].lower()
        if (('permitted use' in text_use) or ('use as' in text_use)):
          use_test.append(text_use)
        i+=1
      if use_test ==  []:
        use_test.append(find_use(filepath_collection[rows]))
      df_cleaned_trimmed.loc[rows,'Use'] = use_test

      # Premise
      premise_test = []
      df_premise = df_cleaned_trimmed['Premise'][rows]
      target_word = ['property known','relating to','premise','related to', 'lr4']
      for keyword in target_word:
        for i in range(len(df_premise)):
          text_premise = df_premise[i].lower()
          if keyword in text_premise:
            text_premise_append = df_premise[i:i+3]
            text_join = ' '.join(text_premise_append)
            premise_test.append(text_join)
            break
          else:
            i+=1
      premise_test = [*set(premise_test)] # Remove duplicate entry
      if premise_test == []:
        premise_test.append(find_premise(filepath_collection[rows]))
      df_cleaned_trimmed.loc[rows,'Premise'] = premise_test
      rows+=1
    # Append two columns landlord and tenant, drop original party column
    df_cleaned_trimmed['Landlord'] = Landlord
    df_cleaned_trimmed['Tenant'] = Tenant
    df_cleaned_trimmed.drop('Parties',axis = 1,inplace=True)
    # Further trimming
    df_cleaned_trimmed = remove_list_df(df_cleaned_trimmed)
    return df_cleaned_trimmed

# Experiment

In [ ]:
# Create a collection containing 5 directories
root_dir = '/content/drive/MyDrive/Dissertation/Experiment_SQL/Lease_text'
file_name = os.listdir(root_dir)
file_path = []
for i in file_name:
  path = root_dir + '/' + i
  file_path.append(path)

In [ ]:
# Experiment
# Create the target dataframe to store
df_cleaned = pd.DataFrame(columns = label)
for i in range(len(file_path)):
  df_cleaned = pipeline(file_path[i],classifier_neo,df_cleaned)
  i+=1
# Trimming
df_cleaned = post_output_transformation(df_cleaned ,file_path)
df_cleaned

<ipython-input-12-03befd5172f4>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_table.sort_values(by=['Index_ref'],inplace=True)
<ipython-input-12-03befd5172f4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_table['Orig_Text'] = text_all
<ipython-input-12-03befd5172f4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

,Tenant_determine,Remain_review_date,Rent_Review_method,Payable_period,Rent_review_upward,yield_up,Full_reinstatement,Tenant_repair_obligation,Tenant_repair_decorate,Structural_alteration,...,assignment,Commence_date,VAT,Garantor,service_proportion,service_landlord,uninsured_risk,service_cap,Landlord,Tenant
0,Yes,17. rent review dates the fifth and tenth anniversary of the term,Other,Yes,Yes,Yes,No,Yes,Yes,Yes,...,Yes,"16. rent commencement date being three months from , and including the term commencement date",Yes,None,Yes,Yes,"contaminate or which may cause an obstruction or damage or could become a source , of injury to any person or otherwise contravenes any statutory requirement , (h) Not to leave the Premises continuously unoccupied for more than one month (save in , the case of damage by an Insured Risk, Uninsured Risk or by some other cause wholly , outside the control of the Tenant) without notifying the Landlord and providing or , paying for such caretaking or security arrangements as the Landlord shall reasonably",Yes,(1) santander (cf trustee) limited and santander (cf trustee,(2) optical center uk limited
1,Yes,1.37 the review dates the first review date means ist april 2015 the review dates means the first review date and the 1st day of april in each of 2020 2025 and 2030. references to a review date are references to any one of the review dates,Other,Yes,Yes,Yes,No,Yes,Yes,No,...,Yes,\x0c1.36 the rent commencement date the rent commencement date means 1° / t 2010,Yes,None,Yes,Yes,"arising out of or in connection with any matter including or relating to the Premises and, 1.11.3 For insurance valuations, 1.12 The Insured Risks, The Insured Risks means the risks of loss or damage by fire storm tempest earthquake, \x0caircraft and articles dropped from aircraft - other than War risks - flood damage and bursting, and overflowing of water pipes and tanks and such other risks whether or not in the nature of",Yes,"(1) vernon john gay, (the landlord) and",CVS Limited
2,Yes,"review dates means 21 november 2018, 21 november 2021and 21",Open Market,Yes,Yes,Yes,No,Yes,Yes,No,...,Yes,None,Yes,None,Yes,Yes,"mfh0257a.doc v 11/13/00 1041 AM, the Building against the Insured Risks and for loss of the Rent for such period, (being not less than three years) as is reasonably required by the Landlord from, 48, Insured Risks, expectant on the determination of the term granted by this Lease;, wall GB.. Landlords. Surveyor.",Yes,"bank hill, woodborough, nottingham ng14 6ef (the landlord);","mundy street, heanor, derbyshire de75 7eb (the tenant)"
3,Yes,on the relevant review date and ending on the day before the quarter,Other,Yes,Yes,Yes,None,Yes,Yes,Yes,...,Yes,None,Yes,None,Yes,Yes,None,No,"lr2.1 landlord’s title number cys (ks limited ., landlord brian alexander tenant cvs (uk) limited anat, ingledale limited and (2) animal hospital limited and (3) malcolm weetman and a tease 9 june 2008 and made between (1) brian alexander and (2) animal hospital limited and (3) malcolm weetman.","lr9.1 tenant’s contractual rights to renew this lease, to acquire the reversion or another lease of the property, or to acquire an interest in other land"
4,Yes,None,Open Market,Yes,Yes,Yes,No,Yes,Yes,Yes,...,Yes,None,Yes,None,Yes,Yes,"(b) against fire lightning, explosion, earthquake, landslip, subsidence, heave, riot,, reasonable excesses and exclusions, 11.2 and to take all necessary steps to make good as soon as possible damage to the property, caused by insured risks except to the extent that the insurance money is not paid because of, the act or default of the Tenant, 41.3 and to give the Tenant at his request once a year particulars of the policy and evidence from",Yes,,"CVS Limited, None"


In [ ]:
# Save to pickle
df_cleaned.to_pickle('/content/drive/MyDrive/Dissertation/Experiment_SQL/Final_output/Exp2.pkl')

# Split into dataframe according to the schema and download to import to database

In [ ]:
# Split into dataframes according to schema
final = pd.read_pickle('/content/drive/MyDrive/Dissertation/Experiment_SQL/Final_output/Exp2.pkl')

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Tenant_determine           5 non-null      object
 1   Remain_review_date         5 non-null      object
 2   Rent_Review_method         5 non-null      object
 3   Payable_period             5 non-null      object
 4   Rent_review_upward         5 non-null      object
 5   yield_up                   5 non-null      object
 6   Full_reinstatement         5 non-null      object
 7   Tenant_repair_obligation   5 non-null      object
 8   Tenant_repair_decorate     5 non-null      object
 9   Structural_alteration      5 non-null      object
 10  non_structural_alteration  5 non-null      object
 11  Premise                    5 non-null      object
 12  Current Tenant             5 non-null      object
 13  Term_period                5 non-null      object
 14  Use           

In [ ]:
# Insepct first row
final.head(1)

In [ ]:
Tenant_index = [0,7,8,14,9,10,22,18,17,26,29,5,31]
Landlord_index = [27,19,6,24,28,30]
Rent_index = [13,15,3,20,2,4,16,21,1]
Property_index = [11]
Other_parties_index = [25,12]

In [ ]:
# Property
property_df = final.iloc[:,Property_index]
property_df['Premise_id'] = property_df.index
# # Move column order
property_df = property_df[['Premise_id'] + [col for col in property_df.columns if col != 'Premise_id']]
property_df.to_csv('Property.csv', index=False)

<ipython-input-107-bea1d1394ca4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_df['Premise_id'] = property_df.index


In [ ]:
# Tenant
tenant_df = final.iloc[:,Tenant_index]
# Premise_id
tenant_df['Premise_id']= property_df['Premise_id']
# Map tenant id
tenant_df['Tenant_id'] = pd.factorize(tenant_df['Tenant'])[0]
# Move column
tenant_df = tenant_df[['Premise_id'] + [col for col in tenant_df.columns if col != 'Premise_id']]
tenant_df = tenant_df[['Tenant_id'] + [col for col in tenant_df.columns if col != 'Tenant_id']]
tenant_df.to_csv('Tenant.csv', index=False)

<ipython-input-108-eb21a70bffe1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tenant_df['Premise_id']= property_df['Premise_id']
<ipython-input-108-eb21a70bffe1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tenant_df['Tenant_id'] = pd.factorize(tenant_df['Tenant'])[0]


In [ ]:
# Landlord
Landlord_df = final.iloc[:,Landlord_index]
# Premise_id
Landlord_df['Premise_id'] = property_df['Premise_id']
# Map landlord id
Landlord_df['Landlord_id'] = pd.factorize(Landlord_df['Landlord'])[0]
# Move column order
Landlord_df = Landlord_df[['Landlord_id'] + [col for col in Landlord_df.columns if col != 'Landlord_id']]
Landlord_df = Landlord_df[['Premise_id'] + [col for col in Landlord_df.columns if col != 'Premise_id']]
Landlord_df.to_csv('Landlord.csv', index=False)

<ipython-input-109-44a306313d05>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Landlord_df['Premise_id'] = property_df['Premise_id']
<ipython-input-109-44a306313d05>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Landlord_df['Landlord_id'] = pd.factorize(Landlord_df['Landlord'])[0]


In [ ]:
# Other parties
Other_party_df = final.iloc[:,Other_parties_index]
# Premise_id
Other_party_df['Premise_id']= property_df['Premise_id']
# Move column
Other_party_df = Other_party_df[['Premise_id'] + [col for col in Other_party_df.columns if col != 'Premise_id']]
Other_party_df.to_csv('Other Parties.csv', index=False)

<ipython-input-110-249250793d1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Other_party_df['Premise_id']= property_df['Premise_id']


In [ ]:
# Rent
Rent_df = final.iloc[:,Rent_index]
# Premise_id
Rent_df['Premise_id']= property_df['Premise_id']
# Move column
Rent_df = Rent_df[['Premise_id'] + [col for col in Rent_df.columns if col != 'Premise_id']]
Rent_df.to_csv('Rent.csv', index=False)

<ipython-input-111-e62a0421e9e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rent_df['Premise_id']= property_df['Premise_id']
